In [17]:
import pandas as pd

#Display entire column
pd.set_option('display.max_colwidth', None)

In [27]:
#Read the downloaded occurence data as a dataframe
ob_data = pd.read_csv("../../Data/rawData/PNW/occurrence.txt",sep='\t')
photo_data = pd.read_csv("../../Data/rawData/PNW/multimedia.txt", sep='\t')

#These two tables can be linked along the 'gbifID' column

In [30]:
#First we drop the irrelevant columns from ob_data
ob_data = ob_data[['gbifID', 'references', 'reproductiveCondition', 'year', 'month', 'day', 'decimalLatitude', 'decimalLongitude', 'species', 'genus', 'family', 'order']]

#Next we drop the rows with null values 
ob_data = ob_data.dropna()

#We will not be studying grasses as their flowers are relatively inconspicuous. Hence we drop the order Poales
ob_data = ob_data[ob_data['order'] != 'Poales']

In [87]:
#Next we want to get image links for each of the following. We write a function which gets an image for each of the above observations under the following conditions: 
#   The image must be hosted by amazon web services 
#   The image must be a jpg or jpeg
#   We only retrieve the first image url for each observation
#   Rather than the original photo, we want the square photo
def get_pix(id):
    rel_pix = photo_data[photo_data['gbifID'] == id]
    try:
        first_pic = rel_pix.iloc[0]
        link = first_pic['identifier']
        #Check if the image is hosted by amazon
        corr_host = ('amazonaws' in link)
        first_ext = link.split('.')[-1]
        corr_extension = first_ext.lower() in ['jpg', 'jpeg']
        if corr_host and corr_extension:
            link_list = link.split('/')
            link_list[-1] = 'square' + '.' + first_ext
            square_link = '/'.join(link_list)
            return square_link
        else:
            return None
    except:
        print('exception')
        return None
    

In [89]:
ordered_pix = ob_data['gbifID'].apply(get_pix)
ob_data['photo_link'] = ordered_pix
ob_data = ob_data.dropna()

In [122]:
#For each observation we download a photo and name it using the gbifID. The method below generates this file name for each observation
def get_extension(id):
    try:
        photo_link = ob_data[ob_data['gbifID'] == id]['photo_link'].iloc[0]
        ext = photo_link.split('.')[-1]
        return ext
    except:
        print(id)



def get_filename(id):
    ext = ob_data[ob_data['gbifID'] == id]['extension'].iloc[0]
    if ext == None:
        return None
    

    return str(id) + '.' + ext


In [124]:
ob_data['extension'] = ob_data['gbifID'].apply(get_extension)
ob_data['file_name'] = ob_data['gbifID'].apply(get_filename)

C:\Users\Sean Haight\AppData\Local\Temp\ipykernel_11564\4210547086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ob_data['extension'] = ob_data['gbifID'].apply(get_extension)
C:\Users\Sean Haight\AppData\Local\Temp\ipykernel_11564\4210547086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ob_data['file_name'] = ob_data['gbifID'].apply(get_filename)


In [129]:
ob_data.head()

,gbifID,references,reproductiveCondition,year,month,day,decimalLatitude,decimalLongitude,species,genus,family,order,photo_link,extension,file_name
4,3456091775,https://www.inaturalist.org/observations/104476974,flowering,2013,5,25,42.698580,-112.352537,Mahonia repens,Mahonia,Berberidaceae,Ranunculales,https://inaturalist-open-data.s3.amazonaws.com/photos/175065922/square.jpeg,jpeg,3456091775.jpeg
29,3859278110,https://www.inaturalist.org/observations/124585118,flowering,2020,7,15,44.473644,-123.537526,Dianthus armeria,Dianthus,Caryophyllaceae,Caryophyllales,https://inaturalist-open-data.s3.amazonaws.com/photos/211178021/square.jpeg,jpeg,3859278110.jpeg
30,3858780054,https://www.inaturalist.org/observations/124647926,fruiting,2022,7,3,48.707505,-122.449975,Vaccinium parvifolium,Vaccinium,Ericaceae,Ericales,https://inaturalist-open-data.s3.amazonaws.com/photos/211294043/square.jpg,jpg,3858780054.jpg
36,4177346397,https://www.inaturalist.org/observations/177602617,fruiting,2023,8,9,45.635351,-122.603530,Crataegus douglasii,Crataegus,Rosaceae,Rosales,https://inaturalist-open-data.s3.amazonaws.com/photos/308917074/square.jpg,jpg,4177346397.jpg
38,2963941302,https://www.inaturalist.org/observations/3735580,flowering,2016,7,23,42.910917,-122.145091,Lupinus sellulus,Lupinus,Fabaceae,Fabales,https://inaturalist-open-data.s3.amazonaws.com/photos/4341165/square.jpg,jpg,2963941302.jpg


In [130]:
ob_data.to_csv('../../Data/processedData/PNW/labeled_PNW_obs.csv', index=False)